In [1]:
import nltk,re,time,umap,collections,os,string,sys,mlflow
sys.path.append(os.path.abspath(".."))
from modules import utils,modeling

from sentence_transformers import SentenceTransformer
import plotly.express as px
import pandas as pd
import pickle

from sklearn.metrics import completeness_score,homogeneity_score, v_measure_score,normalized_mutual_info_score,adjusted_mutual_info_score,silhouette_score,make_scorer

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

from sentence_transformers import models

from hdbscan import HDBSCAN
from sentence_transformers import models
from sklearn.preprocessing import normalize
from transformers import AutoModel, AutoTokenizer
from torch import nn
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

remote_server="http://ec2-18-193-70-141.eu-central-1.compute.amazonaws.com"
client = MlflowClient(remote_server)
mlflow.set_tracking_uri(remote_server)


%matplotlib inline
%load_ext autoreload
%autoreload 2

# UMAP Training 

UMAP as a mainfolf aproximation method performs best on a previously un-seen data when it "pre trained". This notebook fits the UMAP model with the generated emebddings. 


### Load data and the model 

In [2]:
#model = SentenceTransformer('bert-base-wikipedia-sections-mean-tokens')
TOP_N = 1000
TRAIN_N = 50000
MODEL_NAME="cross-en-de-roberta-sentence-transformer"
start_time = time.time()


df = utils.load_text_data("../data/Result_31.csv").head(TRAIN_N).dropna(subset=['text']).sort_values("created_at",ascending=False).drop_duplicates("headline")
df = df[df.text.apply(lambda x: isinstance(x, str))]
#df.drop(["updated_at","site","entities_analyzed"],axis="columns",inplace=True)
print("--- Data Loaded: "+str(df.shape)+" ---")
df.head()

--- Data Loaded: (49379, 8) ---


,kicker,text,headline,seo_title,created_at,kicker_headline_NER,seo_title_NER,text_NER
article_uid,,,,,,,,
3198b3c40190601a35a43f854494c309bd23f2c68d34a331a8544b5701254956,Polizei prüft Verbindung ins Rockermilieu,Alsdorf - Ein 32 Jahre alter Mann ist in Alsdo...,32-Jähriger in Alsdorf erschossen – Täter flüc...,32-Jähriger in Alsdorf (Nordrhein- erschossen ...,2020-11-01 00:11:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
806d166cce198ff3eca5026446f2dfdfa91c0549493a08fecfa33392da86f220,Auf der Reeperbahn,"Es ist 6.10 Uhr am Freitagmorgen, als einer St...",Polizei erwischt Diebe mit Gitarren von Jan Fe...,Jan Fedder: Polizei erwischt Diebe mit seinen ...,2020-10-31 23:23:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
e5c2371084fa38a91f37ebdf10afb5a3e3731a834669250a8282810695b02939,Deutschlands beliebtester Pornostar,Schluss mit Lucy Cat! Sie ist Deutschlands be...,Warum ich nie wieder Lucy Cat sein werde,Porno-Star macht Schluss: Warum ich nie wieder...,2020-10-31 23:23:15,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
e02939be708f60b963b9792988ed27d93fddc2a5bb5b2758fc02986478c64343,TV-Premiere,Für seine neue Late-Night-Show holt sich Stefa...,Ralf Moeller moderiertRaab-Show,TV-Premiere - Ralf Moeller moderiert Raab-Show...,2020-10-31 23:21:16,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."
d86ca23c79f112025c002503646a91ab58ae6b50d79a81b4cae7de329789c5aa,„Hendrik ist bis heute der Mann meines Lebens“,Es sind drei Minuten und 16 Sekunden Gänsehaut...,Inka Bause singt Lied für ihren toten Ex-Mann,Inka Bause singt Lied für toten Ex: „Er ist de...,2020-10-31 23:21:16,"[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":...","[{""entities"":[{""mentions"":[{""text"":{""content"":..."


In [4]:
# load BERT model from Hugging Face
start_time = time.time()

word_embedding_model = models.Transformer(
    'T-Systems-onsite/{}'.format(MODEL_NAME),max_seq_length=250)

print("--- Model & data loaded in %s seconds ---" % (time.time() - start_time))


--- Model & data loaded in 7.582247972488403 seconds ---


In [5]:

df['_text'] = df['headline']+" "+df['kicker'] + ". " + df["text"]#.apply(utils.remove_seo_title_marker)
df['_text'] = df['_text'].astype(str)

pdf = utils.preprocess_articles_for_bert(df,"text",lower=True)
#pdf_mix = utils.preprocess_text(df,col="mix")
#pdf_mix[0][:250]
for nr in  range(50,55):
    print(df.text.values[nr][:100],"\n",pdf[nr][:50])
    print()


Wieder kein Sieg für Aue! Gegen Kiel reicht es für den FC Erzgebirge trotz 75-minütiger Überzahl nur 
 wieder kein sieg für aue. gegen kiel reicht es für

Paris – Nach dem Messerangriff in Nizza mit drei Toten laufen die Ermittlungen auf Hochtouren.  Seit 
 paris nach dem messerangriff in nizza mit drei tot

Geht doch! Dynamo Dresden hat den SV Meppen bei dessen erstem Gastspiel im Harbig-Stadion mit 3:0 ab 
 geht doch. dynamo dresden hat den sv meppen bei de

Real ist wieder in der Spur. Nach dem Last-Minute-2:2 in Gladbach unter der Woche siegen die Königli 
 real ist wieder in der spur. nach dem last-minute-

Arbeitssieg für Fußball-Ästhet Pep Guardiola (49). Mit Manchester City erkämpfte sich der Ex-Bayern- 
 arbeitssieg für fußball-ästhet pep guardiola. mit 



# Load pre calculated 100k Embeddings 

In [6]:

start_time = time.time()

path_to_embeddings ='../models/{}/Top-{}k_articles_embeddings.npy'.format(MODEL_NAME,str(TRAIN_N)[:-3])
path_to_embeddings_with_ids = '../models/{}/Top-{}k_articles_embddings_ids.npy'.format(MODEL_NAME,str(TRAIN_N)[:-3])

if not os.path.exists(path_to_embeddings):
    
    embeddings = modeling.get_sentence_embeddings(pdf,word_embedding_model)
    
    with open(path_to_embeddings, 'wb') as f:
        np.save(f, embeddings)

    with open(path_to_embeddings_with_ids, 'wb') as f:    
        np.save(f, list(zip( df.index.to_numpy(),embeddings)))
else:
    embeddings= np.load(path_to_embeddings)

print("--- %s seconds ---" % (time.time() - start_time))


--- Embedding dimension 768
--- 49379 Documnets encoded 11883.302502155304 seconds ---


FileNotFoundError: [Errno 2] No such file or directory: '../models/cross-en-de-roberta-sentence-transformer/Top-50k_articles_embeddings.npy'

In [7]:
with open(path_to_embeddings, 'wb') as f:
    np.save(f, embeddings)

with open(path_to_embeddings_with_ids, 'wb') as f:    
    np.save(f, list(zip( df.index.to_numpy(),embeddings)))

/Users/tomasztkaczyk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
viz_n_neighbors= 19
min_dist=0.01
start_time = time.time()

fitted_umap_viz = umap.UMAP(n_neighbors=viz_n_neighbors,min_dist=min_dist ,n_components=2, random_state=0, metric='cosine').fit(embeddings)

pkl_filename = "umap_viz_{}_{}-neighbors_{}-min-dist.pkl".format(str(TRAIN_N)[:3],viz_n_neighbors,min_dist)
with open("../models/{}/{}".format(MODEL_NAME,pkl_filename), 'wb') as file:
    pickle.dump(fitted_umap_viz, file)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
n_neighbors = 19
n_components = 32
start_time = time.time()

fitted_umap_clustering = umap.UMAP(n_neighbors=n_neighbors, min_dist=0.01, random_state=0, metric='cosine',
                                       n_components=n_components).fit(embeddings)

pkl_filename = "umap_{}k_{}-neighbors_{}-comps.pkl".format(str(TRAIN_N)[:3],n_neighbors,n_components)
with open("../models/{}/{}".format(MODEL_NAME,pkl_filename), 'wb') as file:
    pickle.dump(fitted_umap_clustering, file)
    
print("--- Model stored.  %s seconds ---" % (time.time() - start_time))

### Interactive tool to find best paramaters for visualization 

In [ ]:


@interact(x=(2,100,1), y=(0.0,1.0,0.010))
def g(x, y,seaborn= True):
    print(x,y)
    result = modeling.umap_for_viz(embeddings,df,n_neighbors=x,min_dist=y)
    if not seaborn:

        fig = px.scatter(result, x="x", y="y",hover_name="headline",hover_data=["created_at"],animation_group="created_at")
        fig["layout"].pop("updatemenus")
        fig.update_traces(marker=dict(size=12,line=dict(width=.35, color='DarkSlateGrey')),
                          selector=dict(mode='markers'))
        fig.update_layout(height=1200)
        return fig.show()
    else:
        plt.figure(figsize=(40, 15))
        return sns.scatterplot(data=result, x="x", y="y")


### Grid Search to find the best UMAP parametrs for vizualisation 

In [91]:
from sklearn.cluster import DBSCAN, KMeans
from sklearn import pipeline


parameters = {"n_neighbors":range(2,64,2),
              "min_dist":(0.0001,0.02, 0.04, 0.16, 0.32, 0.64,0.69, 0.71, 0.73, 0.75, 0.77, 0.79, 0.81, 0.83, 0.85, 1),
              "n_components":[2], 
              "random_state":[0], 
              "metric":['cosine']
          }

#mlflow.log_metric(key="silhuette_score", value=silhouette_score(emb,cluster_labels))
_umap = umap.UMAP()
X = utils.load_labeled_data()
X.drop(["site","entities_analyzed","updated_at"],inplace=True,axis="columns")

emb_ids=np.load("../models/bert-german-dbmdz-uncased-sentence-stsb/Top-100k_articles_embddings_ids.npy",allow_pickle=True)
emb_df = pd.DataFrame(emb_ids,columns=['article_id',"embedding"]).set_index("article_id")
emb = X.join(emb_df,how='left').embedding.to_numpy()
emb = np.stack(emb, axis=0)

parameters = {"umap__n_neighbors":range(2,64,2),
              "umap__min_dist":(0.0001,0.02, 0.04, 0.16, 0.32, 0.64,0.69, 0.71, 0.73, 0.75, 0.77, 0.79, 0.81, 0.83, 0.85, 1),
              "umap__n_components":[2], 
              "umap__random_state":[0], 
              "umap__metric":['cosine']
              
          }


clusterer = KMeans(n_clusters=20,n_init=25)
pipeline = pipeline.Pipeline([
    ('umap', _umap),
    ('model',clusterer)])

grid = GridSearchCV(pipeline, cv=2, n_jobs=-1, param_grid=parameters ,scoring=make_scorer(silhouette_score)
)


grid.fit(emb,[[x] for x in X.labels.values])
grid.best_params_

{'umap__metric': 'cosine',
 'umap__min_dist': 0.0001,
 'umap__n_components': 2,
 'umap__n_neighbors': 58,
 'umap__random_state': 0}

In [104]:
pdf[pdf['_text'].isna() == True]

TypeError: list indices must be integers or slices, not str